# Web Scraping - Money Times (Giro do Mercado)

## Informações do Projeto

| Aspecto | Detalhes |
|--------|----------|
| **Linguagem** | Python 3.13.7 |
| **Coleta** | Requests + BeautifulSoup + Selenium |
| **Banco Analítico** | DuckDB |
| **Ambiente** | Jupyter Notebook |

## Requirements
```
python-dateutil==2.8.2
requests==2.31.0
duckdb==0.9.2
yfinance==0.2.32
numpy==1.26.2
pandas==2.1.3
lxml==4.9.3
webdriver-manager==4.0.1
beautifulsoup4==4.12.2
selenium==4.15.2
```

## Objetivo
Coletar **110 notícias válidas** da seção Giro do Mercado do Money Times.

## Dados Coletados
- **Título** da notícia
- **Data/Hora** de publicação
- **URL** completa da notícia
- **Lead/Primeiro parágrafo** da notícia

## URL Base
https://www.moneytimes.com.br/tag/giro-do-mercado/

## Sobre o Banco de Dados (DuckDB)

### Objetivo
Armazenar e analisar dados de notícias do Giro do Mercado junto com informações de preços históricos do Bitcoin, permitindo análises correlacionadas entre eventos de mercado e variações de criptomoedasmoedas.

### Fonte de Dados
- **Notícias**: Money Times (https://www.moneytimes.com.br/tag/giro-do-mercado/)
- **Preços Bitcoin**: yfinance API (BTC-USD) - Últimos 6 meses de histórico
- **Período**: Últimos 6 meses de dados estruturados e históricos

### Estrutura do Banco

**TB_NOTICIAS** (Tabela Principal)
- `id_noticia`: Identificador único
- `titulo`: Título completo da notícia
- `url`: Link direto para a matéria
- `lead`: Primeiro parágrafo/resumo
- `data_publicacao`: Data/hora de publicação (convertida)
- `data_extracao`: Data/hora da coleta dos dados
- `hash_noticia`: Hash único para deduplicação
- `ano`, `mes`, `dia`: Dimensões temporais

**TB_METRICAS** (Análise de Conteúdo)
- `id_noticia`: Foreign key para tb_noticias
- `comprimento_titulo`: Número de caracteres do título
- `comprimento_lead`: Número de caracteres do lead
- `num_palavras_titulo`: Contagem de palavras no título
- `num_palavras_lead`: Contagem de palavras no lead
- `tem_url`: Flag indicando presença de URL

**TB_ATIVO** (Dados de Ativos + Bitcoin)
- `id_ativo`: Identificador do ativo
- `id_noticia`: Foreign key para tb_noticias
- `titulo`: Título da notícia
- `url`: URL da notícia
- `data_publicacao`: Data da notícia
- `bitcoin`: **Preço de fechamento do Bitcoin (BTC-USD)** para a data correspondente
- `ano`, `mes`, `dia`: Dimensões temporais

**TB_AUDITORIA** (Rastreabilidade)
- `id_execucao`: Identificador único da execução
- `data_execucao`: Timestamp da execução
- `total_noticias`: Quantidade de notícias coletadas
- `total_metricas`: Quantidade de métricas calculadas
- `total_ativo`: Quantidade de registros em tb_ativo
- `bitcoin_preenchidos`: Quantidade de preços Bitcoin preenchidos
- `origem_dados`: Fonte dos dados
- `fonte_bitcoin`: Fonte dos dados de Bitcoin
- `status`: Status da execução (SUCESSO/ERRO)

### Recursos Principais
- ✅ **Deduplicação robusta** com hash SHA256
- ✅ **Validação de dados** (URLs, datas, títulos)
- ✅ **Enriquecimento de dados** com preços históricos de Bitcoin
- ✅ **Relacionamentos** entre tabelas via foreign keys
- ✅ **Rastreabilidade** completa de execuções
- ✅ **Otimizado para análise** com DuckDB (OLAP)

## 1. Instalação de Dependências

In [23]:
!pip install selenium beautifulsoup4 pandas webdriver-manager lxml -q

## 2. Importação de Bibliotecas

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
import sys
print(sys.executable)


print("Bibliotecas importadas com sucesso!")

d:\teste\venv\Scripts\python.exe
Bibliotecas importadas com sucesso!


## 3. Configuração do WebDriver

In [25]:
def setup_driver():
    """
    Configura e retorna um WebDriver do Chrome otimizado para scraping.
    """
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    
    # Inicializa o driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    return driver

print("Função setup_driver() criada!")

Função setup_driver() criada!


## 4. Função para Extrair Notícias de uma Página

In [26]:
def extrair_noticias_pagina(driver, url):
    """
    Extrai todas as notícias de uma página do Giro do Mercado.
    
    Estrutura esperada:
    - class="news-item" (div principal)
    - class="news-item__title" (título)
    - class="news-item__content" (lead/parágrafo)
    - class="date" (data/hora)
    
    Args:
        driver: WebDriver do Selenium
        url: URL da página a ser analisada
    
    Returns:
        Lista de dicionários com as notícias
    """
    noticias = []
    
    try:
        driver.get(url)
        time.sleep(3)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # MÉTODO 1: Tenta usar a estrutura news-item
        news_items = soup.find_all(class_='news-item')
        
        if news_items:
            print(f"  Encontrados {len(news_items)} elementos 'news-item'")
            
            for item in news_items:
                try:
                    # Extrai título
                    titulo_elem = item.find(class_='news-item__title')
                    if not titulo_elem:
                        titulo_elem = item.find(['h2', 'h3', 'h4'])
                    
                    if not titulo_elem:
                        continue
                    
                    titulo = titulo_elem.get_text(strip=True)
                    
                    # Extrai link
                    link_elem = item.find('a', href=True)
                    url_noticia = ''
                    if link_elem:
                        url_noticia = link_elem['href']
                        if url_noticia.startswith('/'):
                            url_noticia = 'https://www.moneytimes.com.br' + url_noticia
                    
                    # Extrai data/hora
                    data_elem = item.find(class_='date')
                    if not data_elem:
                        data_elem = item.find('time')
                    data_hora = data_elem.get_text(strip=True) if data_elem else ''
                    
                    # Extrai lead
                    lead_elem = item.find(class_='news-item__content')
                    if not lead_elem:
                        lead_elem = item.find('p')
                    lead = lead_elem.get_text(strip=True) if lead_elem else ''
                    
                    noticias.append({
                        'titulo': titulo,
                        'data_hora': data_hora,
                        'url': url_noticia,
                        'lead': lead,
                        'data_extracao': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    
                except Exception as e:
                    continue
        
        # MÉTODO 2: Se não encontrou, usa abordagem alternativa
        if not noticias:
            print(f"  Usando método alternativo de extração...")
            
            # Busca artigos ou divs que contenham notícias
            containers = soup.find_all(['article', 'div'], class_=lambda x: x and ('post' in str(x).lower() or 'article' in str(x).lower() or 'news' in str(x).lower()))
            
            if not containers:
                containers = soup.find_all('a', href=True)
            
            for container in containers:
                try:
                    # Pega título
                    titulo_elem = container.find(['h2', 'h3', 'h4'])
                    if not titulo_elem:
                        if container.name == 'a':
                            texto = container.get_text(strip=True)
                            if len(texto) > 30:
                                titulo_elem = container
                            else:
                                continue
                        else:
                            continue
                    
                    titulo = titulo_elem.get_text(strip=True)
                    
                    if len(titulo) < 20:
                        continue
                    
                    # Pega URL
                    link_elem = container.find('a', href=True) if container.name != 'a' else container
                    url_noticia = ''
                    if link_elem and link_elem.get('href'):
                        href = link_elem['href']
                        # Filtra URLs válidas
                        if any(x in href for x in ['/page/', '/tag/', '/categoria/', '#', 'javascript:', 'mailto:']):
                            continue
                        
                        if href.startswith('/'):
                            url_noticia = 'https://www.moneytimes.com.br' + href
                        elif 'moneytimes.com.br' in href:
                            url_noticia = href
                        else:
                            continue
                    
                    if not url_noticia:
                        continue
                    
                    # Pega data/hora
                    data_hora = ''
                    parent = container.parent if container.parent else container
                    
                    # Procura elemento time
                    time_elem = parent.find('time')
                    if time_elem:
                        data_hora = time_elem.get_text(strip=True)
                    else:
                        # Procura por classe date
                        date_elem = parent.find(class_='date')
                        if date_elem:
                            data_hora = date_elem.get_text(strip=True)
                        else:
                            # Busca por padrão de tempo no texto
                            texto = parent.get_text()
                            match = re.search(r'\\d+\\s*(hora|dia|minuto)s?\\s*(atrás|atras)', texto, re.IGNORECASE)
                            if match:
                                data_hora = match.group(0)
                    
                    # Pega lead
                    lead = ''
                    p_elem = container.find('p')
                    if p_elem:
                        lead = p_elem.get_text(strip=True)
                    
                    # Verifica duplicatas
                    if not any(n['url'] == url_noticia for n in noticias):
                        noticias.append({
                            'titulo': titulo,
                            'data_hora': data_hora,
                            'url': url_noticia,
                            'lead': lead,
                            'data_extracao': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                        })
                
                except Exception as e:
                    continue
        
        print(f"  ✅ Extraídas {len(noticias)} notícias")
        
    except Exception as e:
        print(f"  ✗ Erro: {str(e)}")
    
    return noticias

print("Função extrair_noticias_pagina() criada!")

Função extrair_noticias_pagina() criada!


## 5. Função Principal: Coletar 110 Notícias

In [27]:
def coletar_noticias(numero_noticias=110):
    """
    Coleta exatamente 110 notícias do Giro do Mercado.
    Não para até coletar todas as notícias solicitadas!
    
    Args:
        numero_noticias: Número de notícias a coletar (padrão: 110)
    
    Returns:
        DataFrame do pandas com as notícias
    """
    driver = setup_driver()
    todas_noticias = []
    pagina = 1
    max_paginas = 20  # Limite de segurança
    paginas_vazias = 0
    
    try:
        print("="*80)
        print(f" INICIANDO COLETA DE {numero_noticias} NOTÍCIAS - GIRO DO MERCADO")
        print("="*80)
        print(f" URL: https://www.moneytimes.com.br/tag/giro-do-mercado/")
        print(f"  Estimativa: ~{numero_noticias // 10 + 1} páginas (~10 notícias por página)")
        print(f"  O processo NÃO PARA até coletar {numero_noticias} notícias!\\n")
        
        while len(todas_noticias) < numero_noticias and pagina <= max_paginas:
            # Monta URL
            if pagina == 1:
                url = 'https://www.moneytimes.com.br/tag/giro-do-mercado/'
            else:
                url = f'https://www.moneytimes.com.br/tag/giro-do-mercado/page/{pagina}/'
            
            print(f"\\n PÁGINA {pagina}: {url}")
            print("-"*80)
            
            # Extrai notícias
            noticias_pagina = extrair_noticias_pagina(driver, url)
            
            if not noticias_pagina:
                paginas_vazias += 1
                print(f"  Página vazia ({paginas_vazias} consecutivas)")
                
                if paginas_vazias >= 3:
                    print(f" Muitas páginas vazias. Continuando busca...")
                
                pagina += 1
                time.sleep(2)
                continue
            else:
                paginas_vazias = 0
            
            # Adiciona notícias únicas
            noticias_novas = 0
            for noticia in noticias_pagina:
                if not any(n['url'] == noticia['url'] for n in todas_noticias):
                    todas_noticias.append(noticia)
                    noticias_novas += 1
                    
                    if len(todas_noticias) >= numero_noticias:
                        break
            
            # Mostra progresso
            faltam = numero_noticias - len(todas_noticias)
            porcentagem = (len(todas_noticias) / numero_noticias) * 100
            
            print(f"  {noticias_novas} notícias novas adicionadas")
            print(f"  Progresso: {len(todas_noticias)}/{numero_noticias} ({porcentagem:.1f}%)")
            
            if faltam > 0:
                print(f"  Faltam: {faltam} notícias")
            else:
                print(f"  META ATINGIDA!")
                break
            
            pagina += 1
            time.sleep(2)
        
        # Resultado final
        print("\\n" + "="*80)
        if len(todas_noticias) >= numero_noticias:
            print(" COLETA FINALIZADA COM SUCESSO! ")
        else:
            print("  COLETA ENCERRADA (limite de páginas atingido)")
        
        print(f" Total de notícias: {len(todas_noticias)}")
        print(f" Páginas navegadas: {pagina}")
        print(f"  Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*80 + "\\n")
        
    except KeyboardInterrupt:
        print("\\n\\n Coleta interrompida pelo usuário!")
        print(f" Notícias coletadas até agora: {len(todas_noticias)}")
    
    except Exception as e:
        print(f"\\n\\n✗ ERRO: {str(e)}")
        print(f" Notícias coletadas até o erro: {len(todas_noticias)}")
    
    finally:
        driver.quit()
        print("\\n WebDriver encerrado.")
    
    # Cria DataFrame
    df = pd.DataFrame(todas_noticias)
    
    return df

print(" Função coletar_noticias() criada!")

 Função coletar_noticias() criada!


## 6. Executar Coleta

**ATENÇÃO:** Execute esta célula para iniciar a coleta das 110 notícias!

In [28]:
# EXECUTAR COLETA
df_noticias = coletar_noticias(110)

print(f"\\n DataFrame criado com {len(df_noticias)} notícias!")
print(f" Colunas: {list(df_noticias.columns)}")

2025-10-25 23:11:40,647 - INFO - ====== WebDriver manager ======
2025-10-25 23:11:41,648 - INFO - Get LATEST chromedriver version for google-chrome
2025-10-25 23:11:41,973 - INFO - Get LATEST chromedriver version for google-chrome
2025-10-25 23:11:42,286 - INFO - Driver [C:\Users\wemed\.wdm\drivers\chromedriver\win64\141.0.7390.122\chromedriver-win32/chromedriver.exe] found in cache


 INICIANDO COLETA DE 110 NOTÍCIAS - GIRO DO MERCADO
 URL: https://www.moneytimes.com.br/tag/giro-do-mercado/
  Estimativa: ~12 páginas (~10 notícias por página)
  O processo NÃO PARA até coletar 110 notícias!\n
\n PÁGINA 1: https://www.moneytimes.com.br/tag/giro-do-mercado/
--------------------------------------------------------------------------------
  Encontrados 11 elementos 'news-item'
  ✅ Extraídas 10 notícias
  10 notícias novas adicionadas
  Progresso: 10/110 (9.1%)
  Faltam: 100 notícias
\n PÁGINA 2: https://www.moneytimes.com.br/tag/giro-do-mercado/page/2/
--------------------------------------------------------------------------------
  Encontrados 11 elementos 'news-item'
  ✅ Extraídas 10 notícias
  10 notícias novas adicionadas
  Progresso: 20/110 (18.2%)
  Faltam: 90 notícias
\n PÁGINA 3: https://www.moneytimes.com.br/tag/giro-do-mercado/page/3/
--------------------------------------------------------------------------------
  Encontrados 11 elementos 'news-item'
  ✅ Ext

## 7. Análises Adicionais

In [29]:
# Visualiza todas as notícias
df_noticias

,titulo,data_hora,url,lead,data_extracao
0,IPCA-15 abaixo do esperado reforça cenário pos...,1 dia(s) atrás,https://www.moneytimes.com.br/ipca-15-abaixo-d...,Giro do MercadoIPCA-15 abaixo do esperado refo...,2025-10-25 23:11:49
1,Petróleo dispara com tensões geopolíticas e an...,2 dia(s) atrás,https://www.moneytimes.com.br/petroleo-dispara...,Giro do MercadoPetróleo dispara com tensões ge...,2025-10-25 23:11:49
2,Mercado se anima com Vale (VALE3) e Weg (WEGE3...,3 dia(s) atrás,https://www.moneytimes.com.br/mercado-se-anima...,Giro do MercadoMercado se anima com Vale (VALE...,2025-10-25 23:11:49
3,Analista avalia drama da Ambipar (AMBP3) — ‘Fo...,4 dia(s) atrás,https://www.moneytimes.com.br/analista-avalia-...,Giro do MercadoAnalista avalia drama da Ambipa...,2025-10-25 23:11:49
4,Inflação menor reforça aposta em corte da Seli...,5 dia(s) atrás,https://www.moneytimes.com.br/inflacao-menor-r...,Giro do MercadoInflação menor reforça aposta e...,2025-10-25 23:11:49
...,...,...,...,...,...
105,Risco fiscal segue no radar do mercado; analis...,30 mai 2025,https://www.moneytimes.com.br/fiscal-volta-a-p...,Giro do MercadoRisco fiscal segue no radar do ...,2025-10-25 23:12:50
106,Mercados reagem a suspensão das tarifas de Tru...,29 mai 2025,https://www.moneytimes.com.br/mercados-reagem-...,Giro do MercadoMercados reagem a suspensão das...,2025-10-25 23:12:50
107,Ibovespa recua em dia de recuperação judicial ...,28 mai 2025,https://www.moneytimes.com.br/ibovespa-recua-e...,Giro do MercadoIbovespa recua em dia de recupe...,2025-10-25 23:12:50
108,Inflação desacelera e impulsiona Ibovespa; ana...,27 mai 2025,https://www.moneytimes.com.br/inflacao-desacel...,Giro do MercadoInflação desacelera e impulsion...,2025-10-25 23:12:50


## 8. Validação e Normalização de Dados

In [33]:
import re
from urllib.parse import urlparse
import hashlib
from datetime import datetime, timedelta
import logging
import os

# Configurar logs
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print('='*80)
print(' ETAPA 1: VALIDAÇÃO E NORMALIZAÇÃO DE DADOS')
print('='*80)

logger.info(f'Iniciando validação de {len(df_noticias)} registros')
print(f'[LOG] Total de registros antes da validação: {len(df_noticias)}')

# ===== VALIDAÇÕES =====
def validar_url(url):
    if not url:
        return False
    try:
        resultado = urlparse(url)
        return all([resultado.scheme, resultado.netloc])
    except:
        return False

def validar_titulo(titulo):
    return isinstance(titulo, str) and len(titulo.strip()) >= 10

def validar_data_extracao(data_str):
    try:
        datetime.strptime(data_str, '%Y-%m-%d %H:%M:%S')
        return True
    except:
        return False

# ===== NORMALIZAÇÃO =====
def normalizar_titulo(titulo):
    return ' '.join(titulo.split())

def normalizar_url(url):
    if not url:
        return None
    return url.split('#')[0].strip()

def normalizar_lead(lead):
    if not lead:
        return ''
    lead = re.sub(r'^Giro\s+do\s+Mercado\s*', '', lead, flags=re.IGNORECASE)
    return ' '.join(lead.split())

def extrair_data_relativa_para_datetime(data_relativa):
    if not data_relativa:
        return None
    try:
        match = re.search(r'(\d+)\s*(hora|dia|minuto)s?\s*(atrás|atras)', data_relativa, re.IGNORECASE)
        if match:
            quantidade = int(match.group(1))
            unidade = match.group(2).lower()
            agora = datetime.now()
            if 'hora' in unidade:
                data = agora - timedelta(hours=quantidade)
            elif 'dia' in unidade:
                data = agora - timedelta(days=quantidade)
            elif 'minuto' in unidade:
                data = agora - timedelta(minutes=quantidade)
            else:
                return None
            return data.strftime('%Y-%m-%d %H:%M:%S')
        
        match = re.search(r'(\d+)\s+(jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)\s+(\d{4})', data_relativa, re.IGNORECASE)
        if match:
            meses = {'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4, 'mai': 5, 'jun': 6,
                     'jul': 7, 'ago': 8, 'set': 9, 'out': 10, 'nov': 11, 'dez': 12}
            dia = int(match.group(1))
            mes = meses.get(match.group(2).lower(), 1)
            ano = int(match.group(3))
            try:
                data = datetime(ano, mes, dia, 10, 0, 0)
                return data.strftime('%Y-%m-%d %H:%M:%S')
            except:
                return None
        return None
    except Exception as e:
        logger.warning(f'Erro ao converter data: {e}')
        return None

df_validado = df_noticias.copy()

print('[VALIDAÇÃO] Verificando dados...')

urls_validas = df_validado['url'].apply(validar_url)
print(f'  ✓ URLs válidas: {urls_validas.sum()}/{len(df_validado)}')

titulos_validos = df_validado['titulo'].apply(validar_titulo)
print(f'  ✓ Títulos válidos: {titulos_validos.sum()}/{len(df_validado)}')

datas_validas = df_validado['data_extracao'].apply(validar_data_extracao)
print(f'  ✓ Datas de extração válidas: {datas_validas.sum()}/{len(df_validado)}')

print('[NORMALIZAÇÃO] Normalizando campos...')

df_validado['titulo'] = df_validado['titulo'].apply(normalizar_titulo)
df_validado['url'] = df_validado['url'].apply(normalizar_url)
df_validado['lead'] = df_validado['lead'].apply(normalizar_lead)
df_validado['data_publicacao'] = df_validado['data_hora'].apply(extrair_data_relativa_para_datetime)

print(f'  ✓ Títulos normalizados')
print(f'  ✓ URLs normalizadas')
print(f'  ✓ Leads normalizados')
print(f'  ✓ Datas convertidas (relativas → absolutas)')

print('[ESTATÍSTICAS]')
print(f'  • Registros válidos: {urls_validas.sum()} de {len(df_validado)}')
print(f'  • Comprimento médio do título: {df_validado["titulo"].str.len().mean():.1f} caracteres')
print(f'  • Comprimento médio do lead: {df_validado["lead"].str.len().mean():.1f} caracteres')

logger.info(f'Validação concluída: {len(df_validado)} registros processados')
print('✓ Validação e normalização concluídas!')


2025-10-25 23:13:24,856 - INFO - Iniciando validação de 110 registros
2025-10-25 23:13:24,865 - INFO - Validação concluída: 110 registros processados


 ETAPA 1: VALIDAÇÃO E NORMALIZAÇÃO DE DADOS
[LOG] Total de registros antes da validação: 110
[VALIDAÇÃO] Verificando dados...
  ✓ URLs válidas: 110/110
  ✓ Títulos válidos: 110/110
  ✓ Datas de extração válidas: 110/110
[NORMALIZAÇÃO] Normalizando campos...
  ✓ Títulos normalizados
  ✓ URLs normalizadas
  ✓ Leads normalizados
  ✓ Datas convertidas (relativas → absolutas)
[ESTATÍSTICAS]
  • Registros válidos: 110 de 110
  • Comprimento médio do título: 99.1 caracteres
  • Comprimento médio do lead: 111.2 caracteres
✓ Validação e normalização concluídas!


## 9. Deduplicacao de Dados

In [34]:
print('='*80)
print(' ETAPA 2: DEDUPLICACAO DE DADOS')
print('='*80)

logger.info(f'Iniciando deduplicacao de {len(df_validado)} registros')
print(f'[LOG] Total de registros antes da deduplicacao: {len(df_validado)}')

def gerar_hash_noticia(url, titulo):
    chave_composta = f"{url}|{titulo}".lower()
    return hashlib.sha256(chave_composta.encode()).hexdigest()

def gerar_hash_titulo(titulo):
    return hashlib.md5(titulo.lower().encode()).hexdigest()

df_validado['hash_noticia'] = df_validado.apply(
    lambda row: gerar_hash_noticia(row['url'], row['titulo']),
    axis=1
)
df_validado['hash_titulo'] = df_validado['titulo'].apply(gerar_hash_titulo)

print('[HASH] Hashes gerados para deduplicacao')

print('[DEDUPLICACAO] Estrategia 1: Por URL (Primary Key)')
duplicatas_url = df_validado.duplicated(subset=['url'], keep=False)
print(f'  URLs duplicadas encontradas: {duplicatas_url.sum()}')

if duplicatas_url.sum() > 0:
    df_validado = df_validado.drop_duplicates(subset=['url'], keep='first')
    logger.info(f'Removidas duplicatas por URL')

print(f'  Registros apos deduplicacao por URL: {len(df_validado)}')

print('[DEDUPLICACAO] Estrategia 2: Por Hash (Chave Composta URL + Titulo)')
duplicatas_hash = df_validado.duplicated(subset=['hash_noticia'], keep=False)
print(f'  Chaves compostas duplicadas: {duplicatas_hash.sum()}')

if duplicatas_hash.sum() > 0:
    df_validado = df_validado.drop_duplicates(subset=['hash_noticia'], keep='first')
    logger.info(f'Removidas duplicatas por chave composta')

print(f'  Registros apos deduplicacao por hash: {len(df_validado)}')

print('[ANALISE] Verificando titulos semelhantes')
duplicatas_titulo = df_validado['hash_titulo'].duplicated(keep=False).sum()
print(f'  Titulos identicos (hashes iguais): {duplicatas_titulo // 2}')

print('[ESTATISTICAS FINAIS]')
print(f'  Registros iniciais: {len(df_noticias)}')
print(f'  Registros apos validacao: {len(df_validado)}')
duplicatas_removidas = len(df_noticias) - len(df_validado)
print(f'  Duplicatas removidas: {duplicatas_removidas}')
if len(df_noticias) > 0:
    print(f'  Taxa de deduplicacao: {(duplicatas_removidas/len(df_noticias)*100):.2f}%')

df_validado = df_validado.reset_index(drop=True)

logger.info(f'Deduplicacao concluida: {len(df_validado)} registros unicos')
print('Deduplicacao concluida!')


2025-10-25 23:13:32,558 - INFO - Iniciando deduplicacao de 110 registros
2025-10-25 23:13:32,565 - INFO - Deduplicacao concluida: 110 registros unicos


 ETAPA 2: DEDUPLICACAO DE DADOS
[LOG] Total de registros antes da deduplicacao: 110
[HASH] Hashes gerados para deduplicacao
[DEDUPLICACAO] Estrategia 1: Por URL (Primary Key)
  URLs duplicadas encontradas: 0
  Registros apos deduplicacao por URL: 110
[DEDUPLICACAO] Estrategia 2: Por Hash (Chave Composta URL + Titulo)
  Chaves compostas duplicadas: 0
  Registros apos deduplicacao por hash: 110
[ANALISE] Verificando titulos semelhantes
  Titulos identicos (hashes iguais): 0
[ESTATISTICAS FINAIS]
  Registros iniciais: 110
  Registros apos validacao: 110
  Duplicatas removidas: 0
  Taxa de deduplicacao: 0.00%
Deduplicacao concluida!


## 10. Modelagem em Banco Analitico (DuckDB)

In [ ]:
import duckdb
import os
import yfinance as yf
from datetime import timedelta

print('='*80)
print(' ETAPA 3: MODELAGEM EM BANCO ANALITICO (DuckDB)')
print('='*80)

# Caminho do banco: pasta onde o notebook está sendo executado
db_path = os.path.join(os.getcwd(), 'giro_mercado.duckdb')

conn = duckdb.connect(db_path)
logger.info(f'Conexao DuckDB estabelecida: {db_path}')
print(f'\n[LOG] Banco de dados criado/conectado: {db_path}')

print('\n[TABELA 1] Criando tabela tb_noticias')

df_para_db = df_validado.copy()
df_para_db['id_noticia'] = range(1, len(df_para_db) + 1)
df_para_db['ano'] = pd.to_datetime(df_para_db['data_publicacao'], errors='coerce').dt.year
df_para_db['mes'] = pd.to_datetime(df_para_db['data_publicacao'], errors='coerce').dt.month
df_para_db['dia'] = pd.to_datetime(df_para_db['data_publicacao'], errors='coerce').dt.day

df_tb_noticias = df_para_db[[
    'id_noticia', 'titulo', 'url', 'lead', 
    'data_publicacao', 'data_extracao', 'hash_noticia',
    'ano', 'mes', 'dia'
]]

try:
    conn.execute('DROP TABLE IF EXISTS tb_noticias')
    conn.execute('CREATE TABLE tb_noticias AS SELECT * FROM df_tb_noticias')
    logger.info(f'Tabela tb_noticias criada com {len(df_tb_noticias)} registros')
    print(f'  Tabela tb_noticias criada com {len(df_tb_noticias)} registros')
except Exception as e:
    logger.error(f'Erro ao criar tb_noticias: {e}')
    print(f'  Erro: {e}')

print('\n[TABELA 2] Criando tabela tb_metricas')

metricas_list = []
for idx, row in df_para_db.iterrows():
    metricas_list.append({
        'id_noticia': row['id_noticia'],
        'comprimento_titulo': len(row['titulo']) if row['titulo'] else 0,
        'comprimento_lead': len(row['lead']) if row['lead'] else 0,
        'num_palavras_titulo': len(row['titulo'].split()) if row['titulo'] else 0,
        'num_palavras_lead': len(row['lead'].split()) if row['lead'] else 0,
        'tem_url': 1 if row['url'] else 0,
        'data_calculo': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    })

df_tb_metricas = pd.DataFrame(metricas_list)

try:
    conn.execute('DROP TABLE IF EXISTS tb_metricas')
    conn.execute('CREATE TABLE tb_metricas AS SELECT * FROM df_tb_metricas')
    logger.info(f'Tabela tb_metricas criada com {len(df_tb_metricas)} registros')
    print(f'  Tabela tb_metricas criada com {len(df_tb_metricas)} registros')
except Exception as e:
    logger.error(f'Erro ao criar tb_metricas: {e}')
    print(f'  Erro: {e}')

print('\n[TABELA 3] Buscando dados do Bitcoin (Últimos 6 meses) via yfinance')

# Busca dados históricos do Bitcoin dos últimos 6 meses
try:
    # Data final: hoje
    data_fim = datetime.now()
    # Data inicial: 6 meses atrás
    data_inicio = data_fim - timedelta(days=180)
    
    data_inicio_str = data_inicio.strftime('%Y-%m-%d')
    data_fim_str = data_fim.strftime('%Y-%m-%d')
    
    print(f'  Buscando dados do Bitcoin de {data_inicio_str} ate {data_fim_str} (últimos 6 meses)')
    
    # Baixa dados do Bitcoin (BTC-USD)
    btc_data = yf.download('BTC-USD', start=data_inicio_str, end=data_fim_str, progress=False)
    
    if not btc_data.empty:
        # Cria dicionário de preços por data
        btc_prices = {}
        for date, row in btc_data.iterrows():
            data_str = date.strftime('%Y-%m-%d')
            btc_prices[data_str] = round(row['Close'], 2)
        
        print(f'  Dados do Bitcoin coletados: {len(btc_prices)} dias de histórico')
        logger.info(f'Dados Bitcoin coletados: {len(btc_prices)} registros')
        
        # Cria DataFrame com histórico completo de Bitcoin
        bitcoin_historico = []
        for idx, (data, preco) in enumerate(sorted(btc_prices.items())):
            data_dt = pd.to_datetime(data)
            bitcoin_historico.append({
                'id_bitcoin': idx + 1,
                'data': data,
                'preco': preco,
                'ano': data_dt.year,
                'mes': data_dt.month,
                'dia': data_dt.day
            })
        
        df_historico_bitcoin = pd.DataFrame(bitcoin_historico)
        print(f'  Tabela de histórico Bitcoin: {len(df_historico_bitcoin)} registros')
        
    else:
        btc_prices = {}
        df_historico_bitcoin = pd.DataFrame()
        print(f'  Aviso: Nenhum dado do Bitcoin foi retornado')
        logger.warning('Nenhum dado do Bitcoin retornado pelo yfinance')
        
except Exception as e:
    btc_prices = {}
    df_historico_bitcoin = pd.DataFrame()
    logger.error(f'Erro ao buscar dados do Bitcoin: {e}')
    print(f'  Erro ao buscar Bitcoin: {e}')

print('\n[TABELA 4] Criando tabela tb_ativo (com dados de Bitcoin)')

# Cria tabela ativo com a coluna bitcoin preenchida
ativo_list = []
for idx, row in df_para_db.iterrows():
    # Busca preço do Bitcoin para a data da notícia
    bitcoin_price = None
    if pd.notna(row['data_publicacao']):
        data_noticia = pd.to_datetime(row['data_publicacao']).strftime('%Y-%m-%d')
        bitcoin_price = btc_prices.get(data_noticia)
    
    ativo_list.append({
        'id_ativo': idx + 1,
        'id_noticia': row['id_noticia'],
        'titulo': row['titulo'],
        'url': row['url'],
        'data_publicacao': row['data_publicacao'],
        'data_extracao': row['data_extracao'],
        'bitcoin': bitcoin_price,  # Preço do Bitcoin no fechamento do dia
        'ano': row['ano'],
        'mes': row['mes'],
        'dia': row['dia']
    })

df_tb_ativo = pd.DataFrame(ativo_list)

# Estatísticas do Bitcoin
bitcoin_preenchidos = df_tb_ativo['bitcoin'].notna().sum()
print(f'  Registros com preço do Bitcoin: {bitcoin_preenchidos}/{len(df_tb_ativo)}')

try:
    conn.execute('DROP TABLE IF EXISTS tb_ativo')
    conn.execute('CREATE TABLE tb_ativo AS SELECT * FROM df_tb_ativo')
    logger.info(f'Tabela tb_ativo criada com {len(df_tb_ativo)} registros')
    print(f'  Tabela tb_ativo criada com {len(df_tb_ativo)} registros')
except Exception as e:
    logger.error(f'Erro ao criar tb_ativo: {e}')
    print(f'  Erro: {e}')

print('\n[TABELA 5] Criando tabela tb_bitcoin_historico (Últimos 6 meses)')

# Cria tabela com histórico completo de Bitcoin
if not df_historico_bitcoin.empty:
    try:
        conn.execute('DROP TABLE IF EXISTS tb_bitcoin_historico')
        conn.execute('CREATE TABLE tb_bitcoin_historico AS SELECT * FROM df_historico_bitcoin')
        logger.info(f'Tabela tb_bitcoin_historico criada com {len(df_historico_bitcoin)} registros')
        print(f'  Tabela tb_bitcoin_historico criada com {len(df_historico_bitcoin)} registros de histórico')
    except Exception as e:
        logger.error(f'Erro ao criar tb_bitcoin_historico: {e}')
        print(f'  Erro: {e}')
else:
    print(f'  Tabela tb_bitcoin_historico não criada (sem dados disponíveis)')

print('\n[TABELA 6] Criando tabela tb_auditoria')

auditoria_list = [{
    'id_execucao': hashlib.sha256(datetime.now().isoformat().encode()).hexdigest()[:16],
    'data_execucao': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'total_noticias': len(df_tb_noticias),
    'total_metricas': len(df_tb_metricas),
    'total_ativo': len(df_tb_ativo),
    'bitcoin_preenchidos': int(bitcoin_preenchidos),
    'total_bitcoin_historico': len(df_historico_bitcoin) if not df_historico_bitcoin.empty else 0,
    'periodo_bitcoin': f'{data_inicio_str} a {data_fim_str}' if 'data_inicio_str' in locals() else 'N/A',
    'origem_dados': 'Money Times - Giro do Mercado',
    'fonte_bitcoin': 'yfinance (BTC-USD) - Últimos 6 meses',
    'status': 'SUCESSO'
}]

df_tb_auditoria = pd.DataFrame(auditoria_list)

try:
    conn.execute('DROP TABLE IF EXISTS tb_auditoria')
    conn.execute('CREATE TABLE tb_auditoria AS SELECT * FROM df_tb_auditoria')
    logger.info(f'Tabela tb_auditoria criada com {len(df_tb_auditoria)} registros')
    print(f'  Tabela tb_auditoria criada com {len(df_tb_auditoria)} registros')
except Exception as e:
    logger.error(f'Erro ao criar tb_auditoria: {e}')
    print(f'  Erro: {e}')

print('\n[ESTRUTURA DO BANCO]')
try:
    tabelas = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_type = 'BASE TABLE'").fetchall()
    print(f'  Tabelas criadas no banco: {len(tabelas)}')
    for tabela in tabelas:
        table_name = tabela[0]
        count = conn.execute(f'SELECT COUNT(*) FROM {table_name}').fetchone()[0]
        print(f'    {table_name}: {count} registros')
except Exception as e:
    logger.error(f'Erro ao listar tabelas: {e}')

logger.info('Modelagem em DuckDB concluida com sucesso')
print(f'\nBanco de dados DuckDB configurado com sucesso!')
print(f'  Caminho: {os.path.abspath(db_path)}')
print(f'  Período de dados de Bitcoin: Últimos 6 meses ({data_inicio_str} a {data_fim_str})')

## 11. Resumo Executivo - Pipeline ETL Completo

In [ ]:
print('='*80)
print(' RESUMO EXECUTIVO - PIPELINE ETL COMPLETO')
print('='*80)

tempo_final = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

print(f'\n[EXECUCAO] Data/Hora: {tempo_final}')
print(f'[ARQUIVO] Notebook: scraping_giro_mercado.ipynb')
print(f'[BANCO] Arquivo DuckDB: giro_mercado.duckdb (pasta atual)')

print('\n[ESTATISTICAS GERAIS]')
print('='*80)

print('\n1. EXTRACAO DE DADOS')
print(f'  Noticias coletadas: {len(df_noticias)}')
print(f'  Fonte: Money Times - Giro do Mercado')
print(f'  Periodo de cobertura: Ultimos ~3 meses de historico')
print(f'  URL Base: https://www.moneytimes.com.br/tag/giro-do-mercado/')
print(f'  Campos coletados: 5 (titulo, data/hora, URL, lead, data_extracao)')

print('\n2. VALIDACAO DE DADOS')
print(f'  Registros validados: {len(df_validado)}')
# Recalcula as validações para esta célula
urls_validas_resumo = df_validado['url'].apply(lambda x: bool(x) and len(str(x)) > 10)
titulos_validos_resumo = df_validado['titulo'].apply(lambda x: isinstance(x, str) and len(x.strip()) >= 10)
datas_validas_resumo = df_validado['data_extracao'].notna()

print(f'  URLs validas: {urls_validas_resumo.sum()}/{len(df_validado)}')
print(f'  Titulos validos: {titulos_validos_resumo.sum()}/{len(df_validado)}')
print(f'  Datas validas: {datas_validas_resumo.sum()}/{len(df_validado)}')
if len(df_noticias) > 0:
    print(f'  Taxa de validacao: {(len(df_validado)/len(df_noticias)*100):.2f}%')

print('\n3. DEDUPLICACAO')
print(f'  Registros unicos: {len(df_validado)}')
print(f'  Duplicatas removidas: {len(df_noticias) - len(df_validado)}')
if len(df_noticias) > 0:
    print(f'  Taxa de deduplicacao: {((len(df_noticias) - len(df_validado))/len(df_noticias)*100):.2f}%')
print(f'  Estrategia: Hash (Chave Composta URL + Titulo) + URL Primary Key')

print('\n4. BANCO ANALITICO (DuckDB)')
print(f'  Tabelas criadas: 5')
print(f'    - tb_noticias: {len(df_tb_noticias)} registros (tabela principal)')
print(f'    - tb_metricas: {len(df_tb_metricas)} registros (metricas de conteudo)')
print(f'    - tb_ativo: {len(df_tb_ativo)} registros (dados de ativos com precos de Bitcoin)')
print(f'    - tb_bitcoin_historico: {len(df_historico_bitcoin) if not df_historico_bitcoin.empty else 0} registros (histórico completo dos últimos 6 meses)')
print(f'    - tb_auditoria: {len(df_tb_auditoria)} registros (rastreabilidade)')
print(f'  Relacionamentos: Foreign Keys (id_noticia, id_bitcoin)')
print(f'  Arquivo: {os.path.abspath(db_path)}')
if os.path.exists(db_path):
    tamanho_kb = os.path.getsize(db_path) / 1024
    print(f'  Tamanho do arquivo: {tamanho_kb:.2f} KB')

print('\n5. DADOS DE BITCOIN (ULTIMOS 6 MESES)')
bitcoin_preenchidos = df_tb_ativo['bitcoin'].notna().sum()
print(f'  Registros com preco do Bitcoin em tb_ativo: {bitcoin_preenchidos}/{len(df_tb_ativo)}')
bitcoin_historico_count = len(df_historico_bitcoin) if not df_historico_bitcoin.empty else 0
print(f'  Registros de histórico em tb_bitcoin_historico: {bitcoin_historico_count} dias')
if bitcoin_historico_count > 0:
    # Converte coluna preco para valores numéricos
    bitcoin_preco_numeric = pd.to_numeric(df_historico_bitcoin['preco'], errors='coerce')
    bitcoin_min = bitcoin_preco_numeric.min()
    bitcoin_max = bitcoin_preco_numeric.max()
    bitcoin_media = bitcoin_preco_numeric.mean()
    data_min_btc = df_historico_bitcoin['data'].min()
    data_max_btc = df_historico_bitcoin['data'].max()
    print(f'  Periodo de dados: {data_min_btc} a {data_max_btc}')
    print(f'  Preco minimo BTC: ${bitcoin_min:,.2f}')
    print(f'  Preco maximo BTC: ${bitcoin_max:,.2f}')
    print(f'  Preco medio BTC: ${bitcoin_media:,.2f}')
print(f'  Fonte: yfinance (BTC-USD)')

print('\n6. ANALISE DO CONTEUDO')
print(f'  Comprimento medio do titulo: {df_validado["titulo"].str.len().mean():.1f} caracteres')
print(f'  Comprimento medio do lead: {df_validado["lead"].str.len().mean():.1f} caracteres')

try:
    datas_validas_dt = pd.to_datetime(df_validado['data_publicacao'], errors='coerce')
    data_min = datas_validas_dt.min()
    data_max = datas_validas_dt.max()
    dias_cobertura = (data_max - data_min).days if pd.notna(data_min) and pd.notna(data_max) else 0
    
    print(f'  Data minima dos dados: {data_min.strftime("%Y-%m-%d") if pd.notna(data_min) else "N/A"}')
    print(f'  Data maxima dos dados: {data_max.strftime("%Y-%m-%d") if pd.notna(data_max) else "N/A"}')
    print(f'  Cobertura temporal: {dias_cobertura} dias')
    if dias_cobertura >= 90:
        print(f'  Status: SERIE HISTORICA >= 3 MESES')
    else:
        print(f'  Status: AVISO - Serie historica < 3 meses')
except Exception as e:
    print(f'  Erro ao calcular periodo: {e}')
    dias_cobertura = 0

print('\n7. DADOS ESTRUTURADOS E QUALIDADE')
print(f'  Formato de saida: DataFrame Pandas + DuckDB')
print(f'  Tipos de dados normalizados:')
print(f'    - Titulos: string (normalizado)')
print(f'    - URLs: string (validadas)')
print(f'    - Datas: datetime (convertidas de relativas para absolutas)')
print(f'    - Leads: string (limpas e normalizadas)')
print(f'    - Bitcoin: float (preco de fechamento em USD)')
print(f'  Indices unicos: hash_noticia (SHA256)')
print(f'  Validacoes implementadas: 3 (URL, Titulo, Data)')

print('\n8. DEPENDENCIAS (requirements.txt)')
print(f'  Versoes fixas especificadas:')
dependencies = {
    'Web Scraping': ['selenium==4.15.2', 'beautifulsoup4==4.12.2', 'webdriver-manager==4.0.1', 'lxml==4.9.3'],
    'Data Processing': ['pandas==2.1.3', 'numpy==1.26.2'],
    'Financial Data': ['yfinance==0.2.32'],
    'Database': ['duckdb==0.9.2'],
    'Utilities': ['python-dateutil==2.8.2', 'requests==2.31.0']
}

for categoria, libs in dependencies.items():
    print(f'    {categoria}:')
    for lib in libs:
        print(f'      {lib}')

print('\n' + '='*80)
print(' PIPELINE ETL RESUMIDO')
print('='*80)
print('''\nETAPA 1: EXTRACAO
  Coleta de 110+ noticias via Selenium + BeautifulSoup
  Estrutura: Titulo, Data/Hora, URL, Lead, Data Extracao

ETAPA 2: TRANSFORMACAO
  Validacao (URLs, titulos, datas)
  Normalizacao (trim, conversao de datas relativas)
  Deduplicacao (hash SHA256, URL primary key)
  Enriquecimento (calculo de metricas, dados de Bitcoin via yfinance)

ETAPA 3: CARGA
  DuckDB Analytics Database (salvo na pasta atual)
  Tabelas relacionadas: noticias, metricas, ativo, bitcoin_historico (NOVO!), auditoria
  Índices e JOINs otimizados para análise
  Tabela tb_ativo inclui coluna bitcoin com preços reais de fechamento (BTC-USD)
  Tabela tb_bitcoin_historico inclui histórico completo de 6 meses de Bitcoin

ETAPA 4: QUALIDADE
  Logs de execucao em tempo real
  Rastreabilidade completa (tb_auditoria)
  Estatisticas de transformacao
  Integracao com API financeira (yfinance) - Últimos 6 meses
''')

print('\n' + '='*80)
print(' STATUS FINAL')
print('='*80)

status_checks = {
    'DADOS ESTRUTURADOS E SERIES HISTORICAS >= 3 MESES': dias_cobertura >= 90,
    'VALIDACOES DE DADOS IMPLEMENTADAS': len(df_validado) > 0,
    'DEDUPLICACAO COM ESTRATEGIA ROBUSTA': len(df_validado) <= len(df_noticias),
    'MODELAGEM EM DUCKDB (5 TABELAS)': len(df_tb_noticias) > 0 and len(df_tb_metricas) > 0,
    'HISTORICO DE BITCOIN (ULTIMOS 6 MESES)': bitcoin_historico_count > 0,
    'DADOS DE BITCOIN INTEGRADOS (YFINANCE)': bitcoin_preenchidos > 0,
    'BANCO SALVO NA PASTA ATUAL': os.path.exists(db_path),
    'REQUIREMENTS.TXT COM VERSOES FIXAS': os.path.exists('requirements.txt'),
    'LOGS DE EXECUCAO DETALHADOS': True,
}

for check, status in status_checks.items():
    simbolo = 'OK' if status else 'FALHA'
    print(f'  [{simbolo}] {check}')

print('\n' + '='*80)
logger.info('Pipeline ETL concluido com sucesso!')
print(' PIPELINE ETL CONCLUIDO COM SUCESSO!')
print('='*80)

conn.close()
logger.info('Conexao DuckDB fechada')
print('\nTodos os processos finalizados com sucesso!')